In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import random
from statistics import mode
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
%config InlineBackend.figure_format = 'retina'
from sklearn.cluster import KMeans
import random
from sklearn.decomposition import PCA

In [22]:
df_1 = pd.read_csv('sentiment_data/eminem.csv', index_col=0)

In [23]:
df_2 = pd.read_csv('sentiment_data/drake.csv', index_col=0)

In [24]:
df_1.polarity.mean()

-0.46078140096618364

In [25]:
df_2.polarity.mean()

0.22544664804469292

In [26]:
df_1.head()

,Unnamed: 0.1,api_path,primary_artist,title,url,song_endpoint,album,release_date,lyrics,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,polarity
0,0,/songs/4182181,Eminem,12 Days of Diss-Mas,https://genius.com/Eminem-12-days-of-diss-mas-...,http://api.genius.com/songs/4182181,None,2018-12-25,"1. ""Hit 'Em Up"" by 2Pac (Ft. Outlawz) 2. ""Diss...",3,3,2,3,1,4,6,1,2,4,-0.9360
1,1,/songs/3043874,Eminem,1996 Underground Freestyle,https://genius.com/Eminem-1996-underground-fre...,http://api.genius.com/songs/3043874,None,1996-02-17,Any rapper saying those kind of rhymes In this...,2,4,0,2,1,12,6,3,2,4,-0.4599
2,2,/songs/202554,Eminem,1997 Rap Olympics,https://genius.com/Eminem-1997-rap-olympics-ly...,http://api.genius.com/songs/202554,None,2018-12-25,I'ma tell you this for your own benefit Your s...,34,7,26,20,6,51,21,16,6,22,-0.9983
3,3,/songs/695536,Eminem,1999,https://genius.com/Eminem-1999-lyrics,http://api.genius.com/songs/695536,None,2009-07-17,"Hi kids, do you like Kim's bullshit? (No!) Wan...",19,4,19,19,7,31,14,19,7,7,-0.9969
4,4,/songs/2463747,Eminem,2004 Tim Westwood Freestyle,https://genius.com/Eminem-2004-tim-westwood-fr...,http://api.genius.com/songs/2463747,None,2018-12-25,I boss how I live Fuck 'em hoe God meant me To...,139,87,107,120,69,236,179,93,58,134,-0.9989


In [27]:
print(df_1.anger.mean())
print(df_2.anger.mean())

19.78743961352657
10.667597765363128


In [28]:
print(df_1.anticipation.mean())
print(df_2.anticipation.mean())

12.978260869565217
12.978260869565217


In [29]:
print(df_1.fear.mean())
print(df_2.fear.mean())

19.243961352657006
7.650837988826815
